# Toronto Neighborhoods Clustering

In [3]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
!pip install beautifulsoup4

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes


In [5]:
page = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(page.content, 'html.parser')
table_contents=[]
table=soup.find('table')

In [8]:
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

In [9]:

df.shape

(103, 3)

In [10]:

!pip install "geocoder==1.38.1"
from geopy.geocoders import Nominatim

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
     |████████████████████████████████| 98 kB 6.1 MB/s  eta 0:00:01


NameError: name 'postal_code' is not defined

In [ ]:
df2=df['PostalCode']+', Toronto, Ontario'
geolocator = Nominatim(user_agent="foursquare_agent")

for row in df2:
    location = geolocator.geocode(row[0])
    df2['long']= location.longitude
    df2['lat']=location.latitude

def coord(adress):
    locator = Nominatim(user_agent="myGeocoder")
    location = locator.geocode(adress)
    while(location is None):
      
        location = locator.geocode(adress)
        
    long= location.longitude
    lat=location.latitude
    return long,lat
    
dfcoord=pd.read_csv('Geospatial_Coordinates.csv')

dfcoord.rename(columns = {'Postal Code':'PostalCode'}, inplace = True)
dfcoord = pd.merge(df, 
                     dfcoord, 
                     on ='PostalCode', 
                     how ='left')

In [ ]:
import geocoder # import geocoder

# initialize your variable to None
lat_lng_coords = None

# loop until you get the coordinates
while(lat_lng_coords is None):
  g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
  lat_lng_coords = g.latlng

latitude = lat_lng_coords[0]

In [14]:
import requests
import pandas as pd 
import numpy as np 
from IPython.display import Image 
from IPython.core.display import HTML 
from pandas.io.json import json_normalize
! pip install folium==0.5.0
import folium # plotting library
import json 
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
print('Libraries imported.')

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
     |████████████████████████████████| 79 kB 6.3 MB/s  eta 0:00:01
  Created wheel for folium: filename=folium-0.5.0-py3-none-any.whl size=76240 sha256=5111d1b616efb35a1846bf767ef9ee4d3d7288f50ed4f418ba765cf84aba6eb9
  Stored in directory: /tmp/wsuser/.cache/pip/wheels/b2/2f/2c/109e446b990d663ea5ce9b078b5e7c1a9c45cca91f377080f8
Successfully built folium
Libraries imported.


In [31]:
CLIENT_ID = '5QOHL2XULVMA51H2BMVMZXHWDK01MI2Z2D4QMX412RNTGO3Q' 
CLIENT_SECRET = 'OLRNKQNUI2IH0GSRXXGKVFBXIDXYEJD2OVXAPVNBPUHS1CA2' 
ACCESS_TOKEN = 'HGKAR3GOQGAKLHSHNQOEC4LFPXPS2CGQ01FO5ES03C5TXFDF'
VERSION = '20180604'
LIMIT = 30

In [32]:

Toronto_data = dfcoord[dfcoord['Borough'].str.contains("Toronto")].reset_index(drop=True)
                    
Toronto_data.head()

NameError: name 'dfcoord' is not defined

In [ ]:
address = 'Toronto, CA'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

In [ ]:
map_Toronto = folium.Map(location=[latitude, longitude], zoom_start=11)


for lat, lng, label in zip(Toronto_data['Latitude'], Toronto_data['Longitude'], Toronto_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Toronto)  
    
map_Toronto

In [ ]:

Toronto_data.shape

In [19]:
ef getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

SyntaxError: invalid syntax (<ipython-input-19-9826466780b7>, line 1)

In [20]:
Toronto_venues = getNearbyVenues(names=Toronto_data['Neighborhood'],
                                   latitudes=Toronto_data['Latitude'],
                                   longitudes=Toronto_data['Longitude']
                                  )

NameError: name 'getNearbyVenues' is not defined

In [21]:
Toronto_venues.head()

NameError: name 'Toronto_venues' is not defined

In [22]:
Toronto_onehot = pd.get_dummies(Toronto_venues[['Venue Category']], prefix="", prefix_sep="")


Toronto_onehot['Neighborhood'] = Toronto_venues['Neighborhood'] 



Toronto_onehot.head()

NameError: name 'Toronto_venues' is not defined

In [23]:
Toronto_grouped = Toronto_onehot.groupby('Neighborhood').mean().reset_index()

NameError: name 'Toronto_onehot' is not defined

In [24]:
from sklearn.cluster import KMeans
kclusters = 5

Toronto_grouped_clustering = Toronto_grouped.drop('Neighborhood', 1)


kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Toronto_grouped_clustering)


kmeans.labels_[0:10]

NameError: name 'Toronto_grouped' is not defined

In [25]:
Toronto_grouped.insert(0, 'Cluster Labels', kmeans.labels_)
Toronto_merged =Toronto_data


Toronto_merged = Toronto_merged.join(Toronto_grouped.set_index('Neighborhood'), on='Neighborhood')

Toronto_merged.head()

NameError: name 'Toronto_grouped' is not defined

In [26]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)


x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]


markers_colors = []
for lat, lon, poi, cluster in zip(Toronto_merged['Latitude'], Toronto_merged['Longitude'], Toronto_merged['Neighborhood'], Toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

NameError: name 'latitude' is not defined

In [27]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 0, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

NameError: name 'Toronto_merged' is not defined

In [28]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 1, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

NameError: name 'Toronto_merged' is not defined

In [29]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 3, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

NameError: name 'Toronto_merged' is not defined